# Accessing Data From The US Census

This notebook contains code to access data from the US Census' API. There are numerous datasets available ([*link*](https://www.census.gov/data/developers/data-sets.html)).

The **American Community Survey** (ACS, [*link*](https://www.census.gov/programs-surveys/acs)) contains various demographic data collected across the country. There are four versions of this survey based on the timeframe of the collected data (*1 to 5 years of data*) and the granularity of location (*city, state, ZIP Code*). The website offers guidelines for when to use which survey ([*link*](https://www.census.gov/programs-surveys/acs/guidance/estimates.html)).

The schedule for the release of the data is located [here](https://www.census.gov/programs-surveys/acs/news/data-releases/2020/release-schedule.html). As of March 12, 2022 the 2019 ACS was the most recent release, which does not account for the effects of the lockdowns. The 2020 survey is planned to be released on March 17, 2022.

The [examples](https://api.census.gov/data/2019/acs/acs5/examples.html) page contains the formatting necessary to make calls and lists the geographic area of data that is available.

In [1]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import requests
import json
from datetime import date
import os
from tqdm import tqdm
from sodapy import Socrata

## I. A Function To Get State Level Data
https://api.census.gov/data/2019/acs/acs1?get=NAME,B01001_001E&for=state:01


You must match the zipcode with the [state code](https://api.census.gov/data/2019/acs/acs1?get=NAME&for=state:*) in order to get the data.

For instance *36* is the code for NY.

In [2]:
#A function for the API call
# https://www.w3schools.com/python/ref_requests_response.asp

def obtain_census_data(year, codes, state):
    state_code_url = 'https://api.census.gov/data/{}/acs/acs1?get=NAME,{}&for=state:{}'.format(year, codes, state)
    state_code_content = requests.get(state_code_url).json()
    return state_code_content

In [3]:
# An example call for median income
obtain_census_data(year = '2019', codes = 'B19326_001E', state = '36')

[['NAME', 'B19326_001E', 'state'], ['New York', '36165', '36']]

The survey's variable list is [here](https://api.census.gov/data/2019/acs/acs1/variables.html), the variable codes are not always the same year to year and survey version to survey version (*i.e. the 5 year and 1 year surveys*). There are over 31,000 variables in this dataset, loading and searching through them may take a while.

In [4]:
census_codes = {
    "Total_Pop": "B01001_001E",
    "Total_Pop_Male": "B01001_002E",
    "Total_Pop_Female": "B01001_026E",
    "Median_Age": "B01002_001E",
    "Median_Age_Male": "B01002_002E",
    "Median_Age_Female": "B01002_003E",
}

In [5]:
inv_census_codes = {v: k for k, v in census_codes.items()}

In [6]:
# An inverted dictionary
inv_census_codes

{'B01001_001E': 'Total_Pop',
 'B01001_002E': 'Total_Pop_Male',
 'B01001_026E': 'Total_Pop_Female',
 'B01002_001E': 'Median_Age',
 'B01002_002E': 'Median_Age_Male',
 'B01002_003E': 'Median_Age_Female'}

In [7]:
#Inverts the dictionary so the columns can be renamed, add keys from the forthcoming index dictionary to this dictionary
inv_census_codes = {v: k for k, v in census_codes.items()}
# inv_census_codes.update({'District_Name':'District_Name', 'CD': 'CD', 'State_Id': 'State_Id','State': 'State', 'CD_Id_Year': 'CD_Id_Year'})

#Creates a string of codes to be used in the API call
columns_url = ''

for key in census_codes:
    columns_url += census_codes[key] + ','
    
columns_url = columns_url[:-1]

In [8]:
obtain_census_data(year = '2019', codes = columns_url, state = '36')

[['NAME',
  'B01001_001E',
  'B01001_002E',
  'B01001_026E',
  'B01002_001E',
  'B01002_002E',
  'B01002_003E',
  'state'],
 ['New York', '19453561', '9450810', '10002751', '39.2', '37.6', '40.8', '36']]